# RobotChef: EggBot
----
Man used to eat raw meat.
Then we discovered fire.

Man used to chase for food.
Then we discovered agriculture and cultivation.

Man used to cook their own food.
Soon we will have robots.

Welcome to the future :)

## Part 1: Setting Up and Testing Arm Control
________________
This first part will import the libraries necessary to establish communication between the KR260 and the robot arm.

The communication will be via Serial/UART/USB and the communication protocol is pretty simple:

- KR260 sends a 'p' followed by a newline ('\n') --> polls the robot arm joint angles

- KR260 commands the arm like this:
                
                  double; double; double; double; int; int\n

The first four doubles indicate the desired shoulder, elbow, wrist rotation, wrist bend angles, respectively. The last two ints indicate turntable speed and claw servo angle, respectively.

In [1]:
import serial
import serial.tools.list_ports
import numpy as np
import time

### Arm Initialization and Control
The function initArm establishes the Serial connection to the Teensy on the robot arm and returns the Serial object.

In [2]:
# Find the Teensy's USB port
def initArm():
    ports = list(serial.tools.list_ports.comports())

    for p in ports:
        if "USB Serial" in p.description:
            print(p.device)
            print("Connected!")
            break

    # Connect to that port
    arm = serial.Serial(port = p.device, baudrate = 115200, timeout = 0.1)
    time.sleep(3)
    arm.reset_input_buffer()
    return arm

This function polls the position of the current position of the robot arm and returns a list containing the current status of the arm.

In [3]:
# Poll the current arm angles
def getCurrPos(arm):
    arm.write(b"p\n")

    # Wait for Teensy's response
    while arm.in_waiting == 0:
        continue
    
    # Grab the data
    decoded = arm.readline().decode('utf-8')
    arm.reset_input_buffer()
    resp = decoded.rstrip(";\r\n").split(";")

    # Convert strings to floats
    curr_pos = np.array(resp).astype(float).tolist()
    return curr_pos

This function directly drives the arm to the desired joint angles.

In [4]:
# Sends commands angles to arm
def driveArm(arm, angles):
    command = (
        str(angles[0]) + "; " 
        + str(angles[1]) + "; " 
        + str(angles[2]) + "; " 
        + str(angles[3]) + "; " 
        + str(angles[4]) + "; "  
        + str(angles[5]) + "\n"
    )
    arm.write(command.encode('ascii'))


### Arm Control Test
This is a simple test that moves the arm to three positions and back to ensure that everything is working as expected.

In [116]:
pos1 = [0, 30, 0, 200, 0, 1200]
pos3 = [110, 120, 0, 150, 0, 1200]
pos2 = [80, 45, 0, 180, 0, 1200]

In [ ]:
arm = initArm()

In [222]:
driveArm(arm, pos1)

In [ ]:
print(getCurrPos(arm))

In [ ]:
driveArm(arm, pos1)
time.sleep(3)
driveArm(arm, pos2)
time.sleep(3)
driveArm(arm, pos3)
time.sleep(3)
driveArm(arm, pos2)
time.sleep(3)
driveArm(arm, pos1)
time.sleep(3)
driveArm(arm, pos2)
time.sleep(3)
driveArm(arm, pos3)
time.sleep(3)
driveArm(arm, pos2)
time.sleep(3)
driveArm(arm, pos1)

In [101]:
arm.close()

## Part 2: Vision
_________________________
Goal of this section is to test the camera and run the standard Yolov5 framework on the CPU to test model functionality and camera operation.

In [1]:
import cv2

### Camera Test
Open the camera and load video

In [ ]:
cap = cv2.VideoCapture(-1)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

We can also take a picture for testing purposes later on.

In [ ]:
%cd /root/jupyter_notebooks
cap2 = cv2.VideoCapture(-1)
ret, frame = cap2.read()

while(True):
    cv2.imshow('img1', frame)
    if cv2.waitKey(1) & 0xFF == ord('y'):
        cv2.imwrite('test.png', frame)
        cv2.destroyAllWindows()
        break
cap2.release()

### Testing non-accelerated Yolov5 with camera input
This just downloads the Yolov5 model from Ultralytics and runs detect.py using custom weights and the camera input.

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd /root/jupyter_notebooks/yolov5
!pip3 install -r requirements.txt

In [ ]:
%cd /root/jupyter_notebooks
!python3 ./yolov5/detect.py --weights ./yolov5best.pt --img 640 --conf 0.25 --source 0 

### Testing DPU-accelerated Yolov5 with camera input

Last time, each frame takes about 3 seconds to process, giving us a total performance of about 0.3 FPS. Let's try using the DPU.

This code is modified from the dpu-yolov3 tutorial

First import the overlay

In [ ]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")

As well as some other libraries we will be using...

In [2]:
import os
import time
import numpy as np
import cv2
import random
import colorsys
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
%matplotlib inline

Load the xmodel file.

In [ ]:
overlay.load_model("yolov5_kr260.xmodel")

Load the anchor list, this is directly pulled from the anchor list found in yolov5m.yaml

In [4]:
anchor_list = [10,13,16,30,33,23,30,61,62,45,59,119,116,90,156,198,373,326]
anchor_float = [float(x) for x in anchor_list]
anchors = np.array(anchor_float).reshape(-1, 2)

Some preprocessing functions, look at comments

In [5]:
'''Get model classification information'''	
import yaml
def get_class(classes_path):
    with open(classes_path) as f:
        data = yaml.safe_load(f)
        classes = data.get('names', [])
        return classes
    
classes_path = "./Egg-Detector-8/data.yaml"
class_names = get_class(classes_path)

In [6]:
num_classes = len(class_names)
hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x: 
                  (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), 
                  colors))
random.seed(0)
random.shuffle(colors)
random.seed(None)

In [7]:
'''resize image with unchanged aspect ratio using padding'''
def letterbox_image(image, size):
    ih, iw, _ = image.shape
    w, h = size
    scale = min(w/iw, h/ih)
    #print(scale)
    
    nw = int(iw*scale)
    nh = int(ih*scale)
    #print(nw)
    #print(nh)

    image = cv2.resize(image, (nw,nh), interpolation=cv2.INTER_LINEAR)
    new_image = np.ones((h,w,3), np.uint8) * 128
    h_start = (h-nh)//2
    w_start = (w-nw)//2
    new_image[h_start:h_start+nh, w_start:w_start+nw, :] = image
    return new_image


'''image preprocessing'''
def pre_process(image, model_image_size):
    image = image[...,::-1]
    image_h, image_w, _ = image.shape
 
    if model_image_size != (None, None):
        assert model_image_size[0]%32 == 0, 'Multiples of 32 required'
        assert model_image_size[1]%32 == 0, 'Multiples of 32 required'
        boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    else:
        new_image_size = (image_w - (image_w % 32), image_h - (image_h % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0) 	
    return image_data

Processes output tensors into something that we can comprehend.

In [8]:
def _get_feats(feats, anchors, num_classes, input_shape):
    num_anchors = len(anchors)
    anchors_tensor = np.reshape(np.array(anchors, dtype=np.float32), [1, 1, 1, num_anchors, 2])
    grid_size = np.shape(feats)[1:3]
    nu = num_classes + 5
    predictions = np.reshape(feats, [-1, grid_size[0], grid_size[1], num_anchors, nu])
    grid_y = np.tile(np.reshape(np.arange(grid_size[0]), [-1, 1, 1, 1]), [1, grid_size[1], 1, 1])
    grid_x = np.tile(np.reshape(np.arange(grid_size[1]), [1, -1, 1, 1]), [grid_size[0], 1, 1, 1])
    grid = np.concatenate([grid_x, grid_y], axis = -1)
    grid = np.array(grid, dtype=np.float32)

    box_xy = (1/(1+np.exp(-predictions[..., :2])) + grid) / np.array(grid_size[::-1], dtype=np.float32)
    box_wh = np.exp(predictions[..., 2:4]) * anchors_tensor / np.array(input_shape[::-1], dtype=np.float32)
    box_confidence = 1/(1+np.exp(-predictions[..., 4:5]))
    box_class_probs = 1/(1+np.exp(-predictions[..., 5:]))
    return box_xy, box_wh, box_confidence, box_class_probs


def correct_boxes(box_xy, box_wh, input_shape, image_shape):
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = np.array(input_shape, dtype = np.float32)
    image_shape = np.array(image_shape, dtype = np.float32)
    new_shape = np.around(image_shape * np.min(input_shape / image_shape))
    offset = (input_shape - new_shape) / 2. / input_shape
    scale = input_shape / new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes = np.concatenate([
        box_mins[..., 0:1],
        box_mins[..., 1:2],
        box_maxes[..., 0:1],
        box_maxes[..., 1:2]
    ], axis = -1)
    boxes *= np.concatenate([image_shape, image_shape], axis = -1)
    return boxes


def boxes_and_scores(feats, anchors, classes_num, input_shape, image_shape):
    box_xy, box_wh, box_confidence, box_class_probs = _get_feats(feats, anchors, classes_num, input_shape)
    boxes = correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = np.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = np.reshape(box_scores, [-1, classes_num])
    return boxes, box_scores

In [9]:
'''Draw detection frame'''
def draw_bbox(image, bboxes, classes):
    """
    bboxes: [x_min, y_min, x_max, y_max, probability, cls_id] format coordinates.
    """
    num_classes = len(classes)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        fontScale = 0.5
        score = bbox[4]
        class_ind = int(bbox[5])
        bbox_color = colors[class_ind]
        bbox_thick = int(0.6 * (image_h + image_w) / 600)
        c1, c2 = (coor[0], coor[1]), (coor[2], coor[3])
        cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)
    return image


def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2-x1+1)*(y2-y1+1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= 0.45)[0]  # threshold
        order = order[inds + 1]

    return keep

In [10]:
def draw_boxes(image, boxes, scores, classes):
    _, ax = plt.subplots(1)
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image_h, image_w, _ = image.shape

    for i, bbox in enumerate(boxes):
        [top, left, bottom, right] = bbox
        width, height = right - left, bottom - top
        center_x, center_y = left + width*0.5, top + height*0.5
        score, class_index = scores[i], classes[i]
        label = '{}: {:.4f}'.format(class_names[class_index], score) 
        color = tuple([color/255 for color in colors[class_index]])
        ax.add_patch(Rectangle((left, top), width, height,
                               edgecolor=color, facecolor='none'))
        ax.annotate(label, (center_x, center_y), color=color, weight='bold', 
                    fontsize=12, ha='center', va='center')
    return ax

In [11]:
def evaluate(yolo_outputs, image_shape, class_names, anchors):
    score_thresh = 0.1
    anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    boxes = []
    box_scores = []
    input_shape = np.shape(yolo_outputs[0])[1 : 3]
    input_shape = np.array(input_shape)*32

    for i in range(len(yolo_outputs)):
        _boxes, _box_scores = boxes_and_scores(
            yolo_outputs[i], anchors[anchor_mask[i]], len(class_names), 
            input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = np.concatenate(boxes, axis = 0)
    box_scores = np.concatenate(box_scores, axis = 0)

    mask = box_scores >= score_thresh
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(len(class_names)):
        class_boxes_np = boxes[mask[:, c]]
        class_box_scores_np = box_scores[:, c]
        class_box_scores_np = class_box_scores_np[mask[:, c]]
        nms_index_np = nms_boxes(class_boxes_np, class_box_scores_np) 
        class_boxes_np = class_boxes_np[nms_index_np]
        class_box_scores_np = class_box_scores_np[nms_index_np]
        classes_np = np.ones_like(class_box_scores_np, dtype = np.int32) * c
        boxes_.append(class_boxes_np)
        scores_.append(class_box_scores_np)
        classes_.append(classes_np)
    boxes_ = np.concatenate(boxes_, axis = 0)
    scores_ = np.concatenate(scores_, axis = 0)
    classes_ = np.concatenate(classes_, axis = 0)

    return boxes_, scores_, classes_

Now we will use VART to run the model on the DPU.

In [12]:
dpu = overlay.runner

In [13]:
inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

In [14]:
shapeIn = tuple(inputTensors[0].dims)

In [15]:
shapeOut0 = (tuple(outputTensors[0].dims))
shapeOut1 = (tuple(outputTensors[1].dims))
shapeOut2 = (tuple(outputTensors[2].dims))

In [16]:
outputSize0 = int(outputTensors[0].get_data_size() / shapeIn[0])
outputSize1 = int(outputTensors[1].get_data_size() / shapeIn[0])
outputSize2 = int(outputTensors[2].get_data_size() / shapeIn[0])

In [17]:
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
output_data = [np.empty(shapeOut0, dtype=np.float32, order="C"), 
               np.empty(shapeOut1, dtype=np.float32, order="C"),
               np.empty(shapeOut2, dtype=np.float32, order="C")]
image = input_data[0]

Cluster finding code (pseudo Kruskal's Algorithm)

In [5]:
# Calculates distance between two points
def dist(p1, p2):
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    return np.sqrt((x2-x1)**2 + (y2 - y1)**2)

def find(parent, i):
    if parent[i] == i:
        return i
    return find(parent, parent[i])

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)

    if rank[xroot] < rank[yroot]:
        parent[xroot] = yroot
    elif rank[xroot] > rank[yroot]:
        parent[yroot] = xroot
    else:
        parent[yroot] = xroot
        rank[xroot] += 1

# Calculate centroid of a set of points
def centroid(pts):
    point_x = [point[0] for point in pts]
    point_y = [point[1] for point in pts]
    
    centroid_x = np.mean(point_x)
    centroid_y = np.mean(point_y)
    
    return (centroid_x, centroid_y)
        
        
# Gets clusters from a set of points using Union Find
def getClusters(pts, thresh):
    
    #print("Pts: ", pts)
    distances = []
    for i in range(len(pts)):
        for j in range(i + 1, len(pts)):
            distance = dist(pts[i], pts[j])
            if (distance <= thresh):
                distances.append((distance, pts[i], pts[j]))
    
    distances_ = sorted(distances, key=lambda x: x[0])
    
    # Initialize parent and rank arrays for union-find
    parent = list(range(len(pts)))
    rank = [0] * len(pts)

    # Perform union-find on nearby points
    for _, point1, point2 in distances_:
        point1_index = np.where((pts == point1).all(axis=1))[0][0]
        point2_index = np.where((pts == point2).all(axis=1))[0][0]
        union(parent, rank, point1_index, point2_index)

    # Find the connected components (clusters)
    clusters = {}
    for i in range(len(pts)):
        root = find(parent, i)
        if root not in clusters:
            clusters[root] = []
        clusters[root].append(i)

    # Only save clusters that are bigger than 5 elements
    new_clusters = {}
    for cluster_root, points_indices in clusters.items():
        if len(points_indices) >= 5:
            new_clusters[cluster_root] = points_indices
            
    eggs = []
    # Get centroids of the clusters
    for cluster_root, points_indices in new_clusters.items():
        eggs.append(centroid([pts[i] for i in points_indices]))
        
    return eggs
    

Our main function.

In [19]:
def runYolo():
    cap = cv2.VideoCapture(-1)
    
    while cap.isOpened():
        ret, input_image = cap.read()
        # input_image = cv2.cvtColor(frame)
        time1 = time.time()
        # Pre-processing
        image_size = input_image.shape[:2]
        image_data = np.array(pre_process(input_image, (640, 640)), dtype=np.float32)
    
        # Fetch data to DPU and trigger it
        image[0,...] = image_data.reshape(shapeIn[1:])
        job_id = dpu.execute_async(input_data, output_data)
        dpu.wait(job_id)

        # Retrieve output data
        conv_out0 = np.reshape(output_data[0], shapeOut0)
        conv_out1 = np.reshape(output_data[1], shapeOut1)
        conv_out2 = np.reshape(output_data[2], shapeOut2)
        yolo_outputs = [conv_out0, conv_out1, conv_out2]

        # Decode output from YOLOv3
        boxes, _, classes = evaluate(yolo_outputs, image_size, class_names, anchors)

        image_h, image_w, _ = input_image.shape
        
        # Init vars for calculating centroids
        egg_pts = []
        count_eggs = 0

        sumX_strainer = 0
        sumY_strainer = 0
        count_strainer = 0

        sumX_handle = 0
        sumY_handle = 0
        count_handle = 0

        for i, bbox in enumerate(boxes):
            [top, left, bottom, right] = bbox
            width, height = right - left, bottom - top
            center_x, center_y = left + width*0.5, top + height*0.5
            class_index = classes[i]
            
            # Grabs the center of each box found
            # For the sake of this demonstration, we assume there is only one strainer and one handle
            # But the number of eggs is unknown, so we will run a slightly different algorithm
            if (class_index == 0):
                egg_pts.append([center_x, center_y])
                count_eggs += 1
            elif (class_index == 1):
                sumX_strainer += center_x
                sumY_strainer += center_y
                count_strainer += 1
            elif (class_index == 2):
                sumX_handle += center_x
                sumY_handle += center_y
                count_handle += 1
        
        # Circles the average of all boxes
        if (count_eggs > 0):
            all_pts = np.array(egg_pts)
            clusters = getClusters(all_pts, 20)
            colorEgg = tuple([color for color in colors[0]])
            for x_coor, y_coor in clusters:
                cv2.circle(input_image, (int(x_coor), int(y_coor)), radius = 5, color = colorEgg, thickness = 1)
        
        if (count_strainer > 0):
            ctrX_Str = int(sumX_strainer / count_strainer)
            ctrY_Str = int(sumY_strainer / count_strainer)
            colorStr = tuple([color for color in colors[1]])
            cv2.circle(input_image, (ctrX_Str, ctrY_Str), radius = 5, color = colorStr, thickness = 1)
        if (count_handle > 0):
            ctrX_Han = int(sumX_handle / count_handle)
            ctrY_Han = int(sumY_handle / count_handle)
            colorHan = tuple([color for color in colors[2]])
            cv2.circle(input_image, (ctrX_Han, ctrY_Han), radius = 5, color = colorHan, thickness = 1)
            
        time2 = time.time()
        fps = 1/(time2-time1)
        fpsStats = 'FPS: {:.2f}'.format(fps)
        cv2.putText(input_image, fpsStats, (0,30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
        cv2.imshow('frame', input_image)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Run the test! We get about 6 FPS, almost a 20x speedup!

In [20]:
runYolo()

## DON'T FORGET TO RELEASE THE DPU WHEN YOU ARE DONE WITH IT!!!!

In [21]:
del overlay
del dpu

# Part 3: Implementing OCR
_______
Reading the buttons on the stove! Due to time constraints, this part is not accelerated on the DPU, though it may be in the near future. TesseractOCR is used because it is CPU-friendly.

In [ ]:
!sudo apt install -y tesseract-ocr

In [ ]:
!pip3 install pytesseract

In [56]:
import cv2
import numpy as np

In [57]:
import pytesseract
from pytesseract import Output

In [100]:
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def thresholding(image):
    return cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)[1]

In [ ]:
cap = cv2.VideoCapture(-1)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while cap.isOpened():
    ret, frame = cap.read()
    
    gray = get_grayscale(frame)
    thresh = thresholding(gray)
    imagem = cv2.bitwise_not(thresh)
    
    d = pytesseract.image_to_data(imagem, output_type=Output.DICT)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if int(d['conf'][i]) > 40:
            (text, x, y, w, h) = (d['text'][i], d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            # don't show empty text
            if text and text.strip() != "":
                imagem = cv2.rectangle(imagem, (x, y), (x + w, y + h), (0, 255, 0), 2)
                imagem = cv2.putText(imagem, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
    
    cv2.imshow('frame', imagem)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [88]:
cap.release()
cv2.destroyAllWindows()

# Part 4: Forward and Inverse Kinematics
----
ROS-less implementation from scratch, here we go!

In [6]:
import math

### Define Robot Arm Features
These constants define the physical dimensions of the robot arm for the inverse kinematics calculations.

In [7]:
# Base height to shoulder pivot in mm
L_ZOFFSET = 311

# Base X-distance offset from shoulder pivot in mm
L_XOFFSET = 124

# Upper arm length in mm (shoulder pivot --> elbow pivot)
L_UARM = 260

# Upper arm Angle offset in deg (Upper arm --> Elbow point)
A_UOFFSET = 31.7

# Forearm length in mm (elbow pivot --> wrist pivot)
L_4ARM = 274

# Hand Length in mm (wrist pivot --> fingers/grabbing location)
L_HAND = 150

# Wrist pivot to camera X distance in mm
CAM_XDIST = 89

# Wrist_pivot to camera Y distance in mm
CAM_YDIST = 63.5

Since I don't have depth sensing capabilities, I will have to cheat a little and predefine approximate heights of my objects.

In [8]:
# Height of Egg (to center) in mm
H_Egg = 35

# Height of Buttons in mm
H_Button = 86

# Height of strainer in mm
H_Strainer = 229

# Height of handle in mm
H_Handle = 280

Time to do some math. Well, quite a lot of it.

In [9]:
RAD2DEG = 180.0 / math.pi
DEG2RAD = math.pi / 180.0
# Calculates inverse law of cosines
# Input triangle side lengths a, b, c
# Output angle value for the angle opposite of side c
def inv_lawOfCosines(a, b, c):
    return ((math.acos(((a**2)+(b**2)-(c**2))/(2*a*b))) * RAD2DEG)

def lawOfCosines(a, b, C):
    return math.sqrt((a**2)+(b**2)-(2*a*b*math.cos(C*DEG2RAD)))

In [10]:
# Inverse Kinematics solver. 
# Input a 2D location in space with respect to the base of the robot
# Input the target device (claw or camera)
# Also input desired camera or claw angle, business end wrt the ground
# Output is an array of joint angles
def IK(loc, device, angle):
    # Reference location (aka where the shoulder joint is)
    ref = [-L_XOFFSET, L_ZOFFSET]
    target = loc
    
    # Compensating desired position based on device
    if (device == 'cam'):
        target[0] = target[0] - (CAM_XDIST * math.cos(angle * DEG2RAD)) - (CAM_YDIST * math.sin(angle * DEG2RAD))
        target[1] = target[1] - (CAM_XDIST * math.sin(angle * DEG2RAD)) + (CAM_YDIST * math.cos(angle * DEG2RAD))
        
    elif (device == 'claw'):
        target[0] -= L_HAND * math.sin(angle * DEG2RAD)
        target[1] += L_HAND * math.cos(angle * DEG2RAD)
    
    # Distance calculation
    v_side = dist(target, ref)
    v_angle = 180 - math.asin((target[1]-L_ZOFFSET) / v_side) * RAD2DEG
    
    # Shoulder angle
    virt_s_angle = v_angle - inv_lawOfCosines(v_side, L_UARM, L_4ARM)
    s_angle = virt_s_angle - A_UOFFSET
    
    # Elbow angle
    virt_e_angle = inv_lawOfCosines(L_4ARM, L_UARM, v_side)
    e_angle = virt_e_angle - A_UOFFSET
    
    # Wrist bend angle
    y_angle = 180 + angle - (virt_e_angle - (virt_s_angle - 90))
    
    if (device == 'cam'):
        y_angle += 90

    return [s_angle, e_angle, 0, y_angle, 0, 1500]
    
# Forward Kinematics solver.
# Input joint angles (pos)
# Also input desired device
# Returns point in space and device angle wrt the ground
def FK(pos, device):
    s_angle = pos[0]
    e_angle = pos[1]
    y_angle = pos[3]
    
    if (device == 'cam'):
        y_angle -= 90
        
    virt_e_angle = e_angle + A_UOFFSET
    virt_s_angle = s_angle + A_UOFFSET
    
    v_side = lawOfCosines(L_4ARM, L_UARM, virt_e_angle)
    v_angle = virt_s_angle + inv_lawOfCosines(v_side, L_UARM, L_4ARM)
    
    angle = y_angle - 180 + (virt_e_angle - (virt_s_angle - 90))
    
    ypos = (math.sin((180 - v_angle) * DEG2RAD) * v_side) + L_ZOFFSET
    
    xpos = (v_side * math.cos((180 - v_angle) * DEG2RAD)) - L_XOFFSET
    
    if (device == 'cam'):
        xpos = xpos + (CAM_XDIST * math.cos(angle * DEG2RAD)) + (CAM_YDIST * math.sin(angle * DEG2RAD))
        ypos = ypos + (CAM_XDIST * math.sin(angle * DEG2RAD)) - (CAM_YDIST * math.cos(angle * DEG2RAD))
        
    elif (device == 'claw'):
        xpos += L_HAND * math.sin(angle * DEG2RAD)
        ypos -= L_HAND * math.cos(angle * DEG2RAD)
        
    return [xpos, ypos, angle]

In [ ]:
arm = initArm()

IK/FK test, drive it to some known position, measure it in real life, and then use FK to report the position back.

In [ ]:
position = [305, 305]
yeet = IK(position, 'cam', 0)
print(yeet)
driveArm(arm, yeet)
time.sleep(2)
hm = getCurrPos(arm)
print(hm)
data = FK(hm, 'cam')
print("Position: [{}, {}]".format(data[0], data[1]))
print("Angle: ", data[2])

In [ ]:
print(getCurrPos(arm))

In [30]:
arm.close()